In [2]:
from micromind import MicroMind, Metric
from micromind.networks import PhiNet
from micromind.utils.parse import parse_arguments

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from dataset import ImageWoof
from matplotlib import pyplot as plt

from torchinfo import summary

batch_size = 64


In [38]:
#importing libraries

import os
import numpy as np
import torch
import time
import torch.utils.data
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import random
import cv2
import glob
from PIL import Image
import ntpath
import os
from tqdm.notebook import tqdm
%matplotlib inline

In [56]:
def START_seed():
    seed = 9
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

lr = 0.1
epochs=50
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_batch_size = 16
test_batch_size = 16

In [57]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 8, 7, 1),
            nn.ReLU(),
            nn.Conv2d(8, 8, 7, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            #####
            nn.Conv2d(8, 16, 5, 1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 5, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            #####
            nn.Conv2d(16, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(
                output_size=8,
            ),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 8 * 8, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        features = torch.flatten(features, start_dim=1)
        out = self.classifier(features)
        return out

    def num_of_params(self):
        total = 0
        for layer_params in self.feature_extractor.parameters():
            total += layer_params.numel()
        for layer_params in self.classifier.parameters():
            total += layer_params.numel()
        return total

    def compute_loss(self, pred, batch):
        return nn.CrossEntropyLoss()(pred, batch[1])

In [91]:
model = CNN(num_classes=10)

def compute_accuracy(pred, batch):
    tmp = (pred.argmax(1) == batch[1]).float()
    return tmp

transform = transforms.Compose(
    [transforms.ToTensor(), 
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.Resize((160, 160), antialias=True)
    ]
)

trainset = ImageWoof(
    root=".", train=True, transform=transform, img_size=160
)
testset = ImageWoof(
    root=".", train=False, transform=transform, img_size=160
)

## split into train, val, test 
print(len(trainset))     
val_size = int(0.1 * len(trainset))
print(val_size)
train_size = len(trainset) - val_size
train, val = torch.utils.data.random_split(trainset, [train_size, val_size])    

trainloader = torch.utils.data.DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=1
)
valloader = torch.utils.data.DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=1
)    
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=1
)

print("Trainset size: ", len(train)//batch_size)
print("Valset size: ", len(val)//batch_size)
print("Testset size: ", len(testset)//batch_size)

acc = Metric(name="accuracy", fn=compute_accuracy)


9025
902
Trainset size:  63
Valset size:  7
Testset size:  30


In [90]:
summary(model, input_size=(batch_size, 3, 160, 160))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [128, 10]                 --
├─Sequential: 1-1                        [128, 32, 8, 8]           --
│    └─Conv2d: 2-1                       [128, 8, 154, 154]        1,184
│    └─ReLU: 2-2                         [128, 8, 154, 154]        --
│    └─Conv2d: 2-3                       [128, 8, 148, 148]        3,144
│    └─ReLU: 2-4                         [128, 8, 148, 148]        --
│    └─MaxPool2d: 2-5                    [128, 8, 74, 74]          --
│    └─Conv2d: 2-6                       [128, 16, 70, 70]         3,216
│    └─ReLU: 2-7                         [128, 16, 70, 70]         --
│    └─Conv2d: 2-8                       [128, 16, 66, 66]         6,416
│    └─ReLU: 2-9                         [128, 16, 66, 66]         --
│    └─MaxPool2d: 2-10                   [128, 16, 33, 33]         --
│    └─Conv2d: 2-11                      [128, 32, 31, 31]         4,640


In [ ]:
batch = next(iter(trainloader))
data = batch[0]
target = batch[1]

target = target.to('cuda:0')

print(batch[0].shape)
print(len(batch))

print(type(batch))

out = m.forward(batch[0].to('cuda:0'))

print(out)

print(out.shape)

print(torch.argmax(out, dim=1))
print(target)

corrects = target == torch.argmax(out, dim=1)

accuracy = torch.sum(corrects) / len(corrects)

print(accuracy)


torch.Size([128, 3, 160, 160])
2
<class 'list'>
tensor([[ 0.0484,  0.0614, -0.0186,  ..., -0.0041,  0.0187,  0.0285],
        [ 0.0484,  0.0614, -0.0185,  ..., -0.0041,  0.0187,  0.0285],
        [ 0.0484,  0.0614, -0.0186,  ..., -0.0041,  0.0187,  0.0285],
        ...,
        [ 0.0484,  0.0614, -0.0185,  ..., -0.0039,  0.0187,  0.0285],
        [ 0.0484,  0.0614, -0.0186,  ..., -0.0040,  0.0187,  0.0285],
        [ 0.0483,  0.0614, -0.0186,  ..., -0.0040,  0.0187,  0.0285]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([128, 10])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:

In [92]:
START_seed()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(m.parameters(), lr=lr) 

In [97]:
# ## Creating training loop
def train(model):
    model.train()
    train_loss = 0
    total = 0
    correct=0
    with tqdm(trainloader, unit="batch") as tepoch:
        for batch_idx, (data, target) in enumerate(tepoch):
            # send to device
            data, target = data.to(device), target.to(device)                
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*data.size(0)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
        print(' train loss: {:.4f} accuracy: {:.4f}'.format(train_loss/(batch_idx+1), 100.*correct/total))

In [100]:
best_accuracy = 0.0
def validate(model):
    global best_accuracy
    model.eval()

    test_loss = 0
    correct = 0
    total = 0

    with tqdm(valloader, unit="batch") as tepoch:
        for batch_idx, (data, target) in enumerate(tepoch):            
            data, target = data.to(device), target.to(device)    
            output = model(data)
            loss = nn.CrossEntropyLoss()(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
        if (100.*correct/total) > best_accuracy:
            print("Saving the best model...")
            best_accuracy = (100.*correct/total)
            torch.save(model.state_dict(), 'best_model.pth')
        print(' val loss: {:.4f} accuracy: {:.4f} best_accuracy: {:.4f}'.format(test_loss/(batch_idx+1), 100.*correct/total, best_accuracy))


In [101]:
model.to(device)
start = time.time()

for epoch in range(0, epochs):
    print("epoch number: {0}".format(epoch))
    train(model)
    validate(model)
end = time.time()
Total_time=end-start
print('Total training and inference time is: {0}'.format(Total_time))

epoch number: 0


  0%|          | 0/64 [00:00<?, ?batch/s]

 train loss: 292.6050 accuracy: 10.4149


  0%|          | 0/8 [00:00<?, ?batch/s]

Saving the best model...
 val loss: 2.3044 accuracy: 11.4191 best_accuracy: 11.4191
epoch number: 1


...

  0%|          | 0/8 [00:00<?, ?batch/s]

 val loss: 2.3044 accuracy: 11.4191 best_accuracy: 11.4191
epoch number: 6


  0%|          | 0/64 [00:00<?, ?batch/s]

 train loss: 292.6050 accuracy: 10.4149


  0%|          | 0/8 [00:00<?, ?batch/s]

In [ ]:
def test_best_model(model, test_loader, criterion, best_model_path):
    # Load the best model
    model.load_state_dict(torch.load(best_model_path))
    model.eval()

    test_loss = 0
    correct = 0
    total = 0

    with tqdm(testloader, unit="batch") as tepoch:
        for batch_idx, (data, target) in enumerate(tepoch):
            data, target = data.to(device), target.to(device)

            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()

            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

        accuracy = 100. * correct / total
        print('Test loss: {:.4f} Accuracy: {:.2f}%'.format(test_loss/(batch_idx+1), accuracy))

# Usage example:

# Assuming you have a model, test_loader, and best_model_path defined

best_model_path = 'best_model.pth'  # Replace with the actual path and filename of the best model

test_best_model(model, test_loader, nn.CrossEntropyLoss(), best_model_path)
